## Homework3

### Tasks (Purchase prediction)

In [1]:
import gzip
import random

In [2]:
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [3]:
data=list(readGz("assignment1/train.json.gz"))
train_data = data[:100000]
valid_data = data[100000:200000]

#### 1. Although we have built a validation set, it only consists of positive samples. For this task we also need examples of user/item pairs that weren't purchased. Build such a set by randomly sampling users and items until you have 100,000 non-purchased user/item pairs. This random sample combined with your 100,000 validation reviews now corresponds to the complete validation set for the purchase prediction task. Evaluate the performance (accuracy) of the baseline model on the validation set you have built (1 mark).

In [4]:
user_item = defaultdict(list)
users = set()
items =set()
for l in data:
    user,item = l['reviewerID'],l['itemID']
    users.add(user)
    items.add(item)
    user_item[user].append(item)

In [5]:
negative_pair = []
users = list(users)
items = list(items)
count = 0
while count < 100000:
    userrandom = random.choice(users)
    itemrandom = random.choice(items)
    pair = ((userrandom,itemrandom))
    if itemrandom in user_item[userrandom]:
            continue
    else:
        negative_pair.append(pair)
    count+=1

In [6]:
user_valid = [x['reviewerID'] for x in valid_data]
item_valid = [x['itemID'] for x in valid_data]

In [7]:
positive_pair = list(zip(user_valid,item_valid))

In [8]:
whole_pair = positive_pair + negative_pair

In [9]:
businessCount = defaultdict(int)
totalPurchases = 0

for l in train_data:
  user,business = l['reviewerID'],l['itemID']
  businessCount[business] += 1
  totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases/2: break

In [10]:
pred = []
for l in whole_pair:
    if l[1] in return1:
        pred.append(1)
    else:
        pred.append(0)

In [11]:
true = []
for i in range(100000):
    true.append(1)
for i in range(100000):
    true.append(0)

In [12]:
g = []
for i in range(len(pred)):
    if pred[i] == true[i]:
        g.append(1)
    else:
        g.append(0)

In [13]:
accuracy = sum(g)/len(g)
print(accuracy)

0.629035


#### 2. The existing 'purchase prediction' baseline just returns True if the item in question is 'popular', using a threshold of the 50th percentile of popularity (totalPurchases/2). Assuming that the `non-purchased' test examples are a random sample of user-purchase pairs, is this particular threshold value the best? If not, see if you can find a better one (and report its performance), or if so, explain why it is the best (1 mark).

In [19]:
def testthreshold(a):
    businessCount = defaultdict(int)
    totalPurchases = 0

    for l in train_data:
      user,business = l['reviewerID'],l['itemID']
      businessCount[business] += 1
      totalPurchases += 1

    mostPopular = [(businessCount[x], x) for x in businessCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0
    for ic, i in mostPopular:
      count += ic
      return1.add(i)
      if count > totalPurchases*a/100: break
    
    pred = []
    for l in whole_pair:
        if l[1] in return1:
            pred.append(1)
        else:
            pred.append(0)
    
    g = []
    for i in range(len(pred)):
        if pred[i] == true[i]:
            g.append(1)
        else:
            g.append(0)
    accuracy = sum(g)/len(g)
    return(a, accuracy)

In [20]:
accuracylist = []
import numpy as np
for i in np.arange(1, 101, 1):
    accuracylist.append(testthreshold(i))

In [21]:
correct = []
for l in accuracylist:
    correct.append(l[1])

In [22]:
correct.index(max(correct))

52

In [23]:
accuracylist[52]

(53, 0.62949)

##### When the threshold is 53rd percentile of popularity, the accuracy is highest which is 0.62963.

#### 3. Users may tend to repeatedly purchase items of the same type. Build a baseline that returns 'True' if a user has purchased an item of the same category before (at least one category in common), or zero otherwise (1 mark).

In [19]:
user_category = defaultdict(list)
business_category = defaultdict(list)
users = set()
businesses = set()
for l in train_data:
    user,business,category= l['reviewerID'],l['itemID'],l['categories']
    users.add(user)
    businesses.add(business)
    user_category[user].append(category)
    business_category[business].append(category)

In [20]:
for i in users:
    y = []
    for j in user_category[i]:
        for h in j:
            y.append(tuple(h))
    user_category[i] = set(y)

In [21]:
for i in businesses:
    y = []
    for j in business_category[i]:
        for h in j:
            y.append(tuple(h))
    business_category[i] = set(y)

In [22]:
predict_f = []
for user,business in whole_pair:
    a = user_category[user]
    b = business_category[business]
    if any (i in a for i in b):
        predict_f.append(1)
    else:
        predict_f.append(0)

In [23]:
h = []
for i in range(len(predict_f)):
    if predict_f[i] == true[i]:
        h.append(1)
    else:
        h.append(0)
print("validation accuracy is " + str(sum(h)/len(h)))

validation accuracy is 0.59297


#### 4. To run our model on the test set, we'll have to use the files `pairs Purchase.txt' to find the review- erID/itemID pairs about which we have to make predictions. Using that data, run the above model and upload your solution to Kaggle. Tell us your Kaggle user name (1 mark). If you've already uploaded a better solution to Kaggle, that's fine too!

In [24]:
predictions = open("assignment1/predictions_Purchase.csv", 'w')
for l in open("assignment1/pairs_Purchase.txt"):
  if l.startswith("reviewerID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if any (i in user_category[u] for i in business_category[i]):
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + ",0\n")

predictions.close()

##### Kaggle username: Sihao Gu

### Tasks (Rating prediction)

#### Let's start by building our training/validation sets much as we did for the first task. This time building a validation set is more straightforward, you can simply use half of the data for validation, and do not need to randomly sample non-purchased users/items.

#### 5. What is the performance of a trivial predictor rating(user, item) = alpha on the validation set, and what is the value of alpha (1 mark)?

In [25]:
allRatings = []
for l in train_data:
  user,business = l['reviewerID'],l['itemID']
  allRatings.append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)

print('alpha is ' + str(globalAverage))

alpha is 4.232


In [26]:
MSE = 0
for l in valid_data:
    MSE += (l['rating'] - globalAverage) ** 2

MSE = MSE/len(train_data)

print('MSE is ' + str(MSE))

MSE is 1.222481119999121


#### 6. Fit a predictor of the form rating(user, item) = alpha + Beta(user) + Beta(item); by fitting the mean and the two bias terms as described in the lecture notes. Use a regularization parameter of lambda = 1. Report the MSE on the validation set (1 mark).

In [27]:
# calculate initial alpha, Beta(user), Beta(item)
allRatings = []
userRatings = defaultdict(list)
businessRatings = defaultdict(list)
for l in train_data:
  user,business = l['reviewerID'],l['itemID']
  allRatings.append(l['rating'])
  userRatings[user].append(l['rating'])
  businessRatings[business].append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
businessAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
for u in businessRatings:
  businessAverage[u] = sum(businessRatings[u]) / len(businessRatings[u])

In [28]:
def iterate(globalAverage, userAverage, businessAverage, lam = 1, \
            threshold = 0.00001):
    switch = 0
    prev_globalAverage = globalAverage
    prev_userAverage = userAverage
    prev_businessAverage = businessAverage
    while switch == 0:
        globalAverage = 0
        for l in train_data:
            user,business = l['reviewerID'],l['itemID']
            globalAverage += (l["rating"] - prev_userAverage[user] - \
                              prev_businessAverage[business])
        globalAverage /= len(train_data)
        
        nbusiness = defaultdict(int)
        for user in userAverage:
            userAverage[user] = 0
        for l in train_data:
            user,business = l['reviewerID'],l['itemID']
            nbusiness[user] += 1
            userAverage[user] += (l["rating"] - globalAverage - \
                                  prev_businessAverage[business])
        for user in userAverage:
            userAverage[user] /= (lam + nbusiness[user])
        
        difference_user = 0
        for i in userAverage:
            difference_user += abs(userAverage[i] - prev_userAverage[i])
            
        nuser = defaultdict(int)
        for business in businessAverage:
            businessAverage[business] = 0
        for l in train_data:
            user,business = l['reviewerID'],l['itemID']
            nuser[business] += 1
            businessAverage[business] += (l["rating"] - globalAverage - \
                                          userAverage[user])
        for business in businessAverage:
            businessAverage[business] /= (lam + nuser[business])
        
        difference_bus = 0
        for i in businessAverage:
            difference_bus += abs(businessAverage[i] - prev_businessAverage[i])        
        
        if abs(globalAverage - prev_globalAverage) < threshold and \
        difference_user < threshold and \
        difference_bus < threshold:
            switch = 1
        else:
            prev_globalAverage = globalAverage
            prev_userAverage = userAverage
            prev_businessAverage = businessAverage
    return globalAverage, userAverage, businessAverage

In [29]:
# Calculate MSE
def MSE(globalAverage_update, userAverage_update, businessAverage_update):
    mse = 0.0
    userlist = []
    businesslist = []
    for i in userAverage_update:
        userlist.append(i)
    for j in businessAverage_update:
        businesslist.append(j)
    for l in valid_data:
        user,business = l['reviewerID'],l['itemID']
        if (user in userlist) & (business in businesslist):
            mse += (l["rating"]-globalAverage_update- \
                    userAverage_update[user]-businessAverage_update[business])**2
        elif (user in userlist) & (business not in businesslist):
            mse += (l["rating"]-globalAverage_update-userAverage_update[user])**2
        elif (user not in userlist) & (business in businesslist):
            mse += (l["rating"]-globalAverage_update-\
                    businessAverage_update[business])**2
        else:
            mse += (l["rating"]-globalAverage_update)**2
    mse /= 100000
    print("MSE on validation data is " + str(mse))

In [30]:
globalAverage_update, userAverage_update, businessAverage_update = iterate(\
            globalAverage,userAverage, businessAverage, lam = 1, threshold = 0.00001)

In [31]:
MSE(globalAverage_update,userAverage_update, businessAverage_update)

MSE on validation data is 1.281118659105882


#### 7. Report the user and item IDs that have the largest and smallest values of beta (1 mark).

In [34]:
# smallest beta in User
sorted(userAverage_update.items(), key = lambda item:item[1])[0]

('U204516481', -2.5517066060072136)

In [35]:
# largest beta in User
sorted(userAverage_update.items(), key = lambda item:item[1])[-1]

('U495776285', 1.486748655993029)

In [36]:
# smallest beta in itemIDs
sorted(businessAverage_update.items(), key = lambda item:item[1])[0]

('I511389419', -2.575677187832548)

In [37]:
# largest beta in itemIDs
sorted(businessAverage_update.items(), key = lambda item:item[1])[-1]

('I809804570', 1.2700396852824163)

#### 8. Find a better value of lambda using your validation set. Report the value you chose, its MSE, and upload your solution to Kaggle by running it on the test data (1 mark).

In [38]:
for lam in [2.5,5,7.5,10]:
    globalAverage_update, userAverage_update, businessAverage_update = iterate(\
        globalAverage, userAverage, businessAverage, lam = lam, threshold = 0.00001)
    print('lam= ' + str(lam))
    MSE(globalAverage_update, userAverage_update, businessAverage_update)

lam= 2.5
MSE on validation data is 1.170350602374201
lam= 5
MSE on validation data is 1.139895608962076
lam= 7.5
MSE on validation data is 1.13809132350018
lam= 10
MSE on validation data is 1.1416031165965732


In [39]:
for lam in [5.5,6,6.5,7]:
    globalAverage_update, userAverage_update, businessAverage_update = iterate(\
        globalAverage, userAverage, businessAverage, lam = lam, threshold = 0.00001)
    print('lam= ' + str(lam))
    MSE(globalAverage_update, userAverage_update, businessAverage_update)

lam= 5.5
MSE on validation data is 1.1386055295339117
lam= 6
MSE on validation data is 1.137923962774243
lam= 6.5
MSE on validation data is 1.1376840308091374
lam= 7
MSE on validation data is 1.137768041725769


In [40]:
for lam in [6.6,6.7,6.8,6.9]:
    globalAverage_update, userAverage_update, businessAverage_update = iterate(\
        globalAverage, userAverage, businessAverage, lam = lam, threshold = 0.00001)
    print('lam= ' + str(lam))
    MSE(globalAverage_update, userAverage_update, businessAverage_update)

lam= 6.6
MSE on validation data is 1.1376780068768086
lam= 6.7
MSE on validation data is 1.1376850951675586
lam= 6.8
MSE on validation data is 1.1377016238720616
lam= 6.9
MSE on validation data is 1.1377306752091179


##### Because when lam=6.6, MSE is minimized. So we choose lam=6.6

In [44]:
allRatings = []
userRatings = defaultdict(list)
businessRatings = defaultdict(list)
for l in valid_data:
  user,business = l['reviewerID'],l['itemID']
  allRatings.append(l['rating'])
  userRatings[user].append(l['rating'])
  businessRatings[business].append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
businessAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
for u in businessRatings:
  businessAverage[u] = sum(businessRatings[u]) / len(businessRatings[u])

In [45]:
def iterate1(globalAverage, userAverage, businessAverage, lam = 1, \
            threshold = 0.00001):
    switch = 0
    prev_globalAverage = globalAverage
    prev_userAverage = userAverage
    prev_businessAverage = businessAverage
    while switch == 0:
        globalAverage = 0
        for l in valid_data:
            user,business = l['reviewerID'],l['itemID']
            globalAverage += (l["rating"] - prev_userAverage[user] - \
                              prev_businessAverage[business])
        globalAverage /= len(valid_data)
        
        nbusiness = defaultdict(int)
        for user in userAverage:
            userAverage[user] = 0
        for l in valid_data:
            user,business = l['reviewerID'],l['itemID']
            nbusiness[user] += 1
            userAverage[user] += (l["rating"] - globalAverage - \
                                  prev_businessAverage[business])
        for user in userAverage:
            userAverage[user] /= (lam + nbusiness[user])
        
        difference_user = 0
        for i in userAverage:
            difference_user += abs(userAverage[i] - prev_userAverage[i])
            
        nuser = defaultdict(int)
        for business in businessAverage:
            businessAverage[business] = 0
        for l in valid_data:
            user,business = l['reviewerID'],l['itemID']
            nuser[business] += 1
            businessAverage[business] += (l["rating"] - globalAverage - \
                                          userAverage[user])
        for business in businessAverage:
            businessAverage[business] /= (lam + nuser[business])
        
        difference_bus = 0
        for i in businessAverage:
            difference_bus += abs(businessAverage[i] - prev_businessAverage[i])        
        
        if abs(globalAverage - prev_globalAverage) < threshold and \
        difference_user < threshold and \
        difference_bus < threshold:
            switch = 1
        else:
            prev_globalAverage = globalAverage
            prev_userAverage = userAverage
            prev_businessAverage = businessAverage
    return globalAverage, userAverage, businessAverage

In [46]:
globalAverage_update, userAverage_update, businessAverage_update = iterate(\
    globalAverage, userAverage, businessAverage, lam = 6.95, threshold = 0.00001)

In [47]:
userlist = []
businesslist = []
for i in userAverage_update:
    userlist.append(i)
for j in businessAverage_update:
    businesslist.append(j)
predictions = open("assignment1/predictions_Rating.csv", 'w')
for l in open("assignment1/pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')    
    if (u in userlist) and (i in businesslist):
        rating=globalAverage_update+userAverage_update[u]+businessAverage_update[i]
        predictions.write(u + '-' + i + ","+str(rating)+"\n")
    elif (u in userlist) and (i not in businesslist):
        rating=globalAverage_update+userAverage_update[u]
        predictions.write(u + '-' + i + ","+str(rating)+"\n")
    elif (u not in userlist) and (i in businesslist):
        rating=globalAverage_update+businessAverage_update[i]
        predictions.write(u + '-' + i + ","+str(rating)+"\n")
    else:
        rating=globalAverage_update
        predictions.write(u + '-' + i + ","+str(rating)+"\n")
predictions.close()